In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import sys
sys.path.append("../..")

In [3]:
import numpy as np

In [4]:
from librep.datasets.har.loaders import (
    MotionSense_BalancedView20HZ
)
from experiments.Experiment_utils import run_experiments_accuracy_f1
from librep.transforms.fft import FFT
from librep.datasets.multimodal import TransformMultiModalDataset

2022-12-22 12:14:14.412986: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2022-12-22 12:14:14.529225: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2022-12-22 12:14:14.529249: I tensorflow/compiler/xla/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.
2022-12-22 12:14:15.310560: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory
2022-

# Experimento 8:
*Aplicar dois tipos de normalização (Scaling e Z-norm) no dado já transformado pela função não paramétrica (p.ex,  no domínio da frequência, caso esta seja a melhor) e verificar para um dataset apenas*

## Pasos a seguir:
- Carregar o dataset MotionSense.
- Aplicar Fourier para deixar MotionSense no dominio da frequencia
- Aplicar a normalização MinMaxScaling
- Aplicar a normalização Znorm
- Comparar acurácias com modelos sem normalização 

## Subexperimento 8.2 - Normalização MinMaxScaling

### MotionSense

In [5]:
# MotionSense Loader
loader = MotionSense_BalancedView20HZ(
    root_dir="../../data/views/MotionSense/balanced_view_20Hz_with_gravity_9.81_acc_standard", 
    download=False
)

In [6]:
train_val, test = loader.load(concat_train_validation=True, label=loader.standard_label)

In [7]:
train_HD = np.array(train_val[:][0])
train_Y = np.array(train_val[:][1])
test_HD = np.array(test[:][0])
test_Y = np.array(test[:][1])

### FFT

In [8]:
fft_transform = FFT(centered = True)
transformer = TransformMultiModalDataset(
    transforms=[fft_transform],
    new_window_name_prefix="fft."
)

In [9]:
train_dataset_fft = transformer(train_val)
test_dataset_fft = transformer(test)

In [10]:
print(train_dataset_fft.X.shape)
print(test_dataset_fft.X.shape)

(4020, 180)
(1020, 180)


In [11]:
print(np.max(train_dataset_fft.X))

797.3368276806895


In [12]:
train_HD = train_dataset_fft.X
train_LD = None
train_Y = train_dataset_fft.y
test_HD = test_dataset_fft.X
test_LD = None
test_Y = test_dataset_fft.y

In [13]:
print(train_HD.shape, train_Y.shape, test_HD.shape, test_Y.shape)

(4020, 180) (4020,) (1020, 180) (1020,)


### MinMaxScaling

In [14]:
from sklearn.preprocessing import MinMaxScaler

scaler = MinMaxScaler()
scaler.fit(train_HD)
train_HD = scaler.transform(train_HD)
test_HD = scaler.transform(test_HD)

### Acurácia e F1

In [15]:
experiments_result_min_max = run_experiments_accuracy_f1(train_HD, train_Y, test_HD, test_Y)

In [16]:
%store experiments_result_min_max

Stored 'experiments_result_min_max' (dict)
